# Converting project to use signac

This tutorial will demonstrate how to turn an existing project into a project tracked by signac, assuming the project uses nested directories to organize the data.


First, create a signac project to import into

In [ ]:
!signac init

We will use the `signac import` command

In [ ]:
!signac import --help

What is the schema of the directory layout?

Let's import this project into signac by writing the appropriate schema_path for the `import` command

In [ ]:
!signac import random_walk_project 'standard_deviation/{standard_deviation:float}/replica_{replica:int}_seed_{seed:int}'


## Converting code to use signac

For this exercise, we will only convert the first two operations (`simulate` and `compute_squared_displacement`), so we have reproduced those parts here.

The main actions involve:

- Get parameters from signac state point (job.sp)
- Replace cases of os.path.join(path, ...) with job.fn(...)

In [ ]:
import signac
import numpy as np
import h5py
import matplotlib.pyplot as plt

In [ ]:
project = signac.get_project()

First, let's find the signac_statepoint.json file and confirm that the import was correct.

In [ ]:
project.find_jobs({'seed': 221141651})

In [ ]:
filter_match

Follow along to convert these variables to load from the job state point

In [ ]:
job = next(iter(filter_match))

In [ ]:
run_steps = 5000
mean = 0

standard_deviation = job.sp.standard_deviation
replica = job.sp.replica
seed = job.sp.seed


Convert the `simulate` operation:

In [ ]:
# Get variables from state point

generator = np.random.default_rng(seed)

# Generate all moves at once since they are independent
moves = generator.normal(mean, standard_deviation, (run_steps, 2))
positions = np.zeros((run_steps + 1, 2), dtype=float)

# Store the results: perform a cumulative sum of all moves starting from the origin
positions[1:] = np.cumsum(moves, axis=0)



# Recall what file saving looked like, but note that the old path will not work.

# with h5py.File(os.path.join(path, "signac_data.h5"), 'w') as hf:
#     hf.create_dataset("positions", data=positions, dtype='f')

Convert the `compute_squared_displacement` operation:

In [ ]:
squared_displacement = np.sum(positions * positions, axis=1)


# Recall what file saving looked like, but note that the old path will not work.

#with h5py.File(os.path.join(path, "signac_data.h5"), 'w') as hf:
#    hf.create_dataset("squared_displacement", data=squared_displacement, dtype='f')

Once you have converted these cells, we'll give an overview of the `project-1.py` template.

# Finish evaluating the workflow in signac-flow

Check status of the project.

- Note that 20/25 jobs have satisfied the `simulated` label
    - Those 5 eligible `simulate` operations show up in the list
- 


In [ ]:
!python project-1.py status

In [ ]:
!python project-1.py run -o simulate -n 2


In [ ]:
!python project-1.py status

In [ ]:
!python project-1.py run -o compute_squared_displacement

In [ ]:
!python project-1.py status

Using the command line interface to run and check status, compute the rest of the workflow.

In [ ]:
!python project-1.py run

Oops! We haven't defined the run_steps for all our jobs!

In [ ]:
for job in project:
    job.doc.run_steps = 5000